In [1]:
from datasets import load_dataset
import numpy as np, evaluate
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers.trainer_utils import IntervalStrategy

In [2]:
# 데이터/메트릭
ds = load_dataset("PolyAI/banking77")
acc = evaluate.load("accuracy"); f1 = evaluate.load("f1")
def compute_metrics(p):
    logits, labels = p
    preds = np.argmax(logits, axis=1)
    return {"accuracy": acc.compute(predictions=preds, references=labels)["accuracy"],
            "f1_macro": f1.compute(predictions=preds, references=labels, average="macro")["f1"]}


Using the latest cached version of the dataset since PolyAI/banking77 couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /Users/jessicahong/.cache/huggingface/datasets/PolyAI___banking77/default/1.1.0/17ffc2ed47c2ed928bee64127ff1dbc97204cb974c2f980becae7c864007aed9 (last modified on Thu Aug 21 22:30:16 2025).


In [3]:
# 토크나이즈
model_name = "distilbert-base-uncased"
tok = AutoTokenizer.from_pretrained(model_name)
def tok_fn(b): return tok(b["text"], truncation=True, padding="max_length", max_length=64)
ds_tok = ds.map(tok_fn, batched=True).rename_column("label","labels")
ds_tok.set_format(type="torch", columns=["input_ids","attention_mask","labels"])


Map:   0%|          | 0/10003 [00:00<?, ? examples/s]

Map:   0%|          | 0/3080 [00:00<?, ? examples/s]

In [4]:
# 모델 + 학습(3 epochs)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=77)
args = TrainingArguments(
    output_dir="./out_distilbert_3e",
    learning_rate=2e-5,
    num_train_epochs=3,                 # ← 3
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    eval_strategy=IntervalStrategy.EPOCH,
    save_strategy=IntervalStrategy.EPOCH,
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    greater_is_better=True,
    report_to=None,
    dataloader_pin_memory=False,
    save_total_limit=2,
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
trainer = Trainer(model=model, args=args,
                  train_dataset=ds_tok["train"], eval_dataset=ds_tok["test"],
                  tokenizer=tok, compute_metrics=compute_metrics)

/var/folders/6y/xtl4b0cx1cs9zrr9n5y814_h0000gn/T/ipykernel_6804/483207701.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model, args=args,


In [6]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro
1,2.219100,1.294205,0.773701,0.748236
2,0.638500,0.588167,0.871429,0.866324
3,0.401300,0.466521,0.894156,0.893645


TrainOutput(global_step=3753, training_loss=1.300201029768633, metrics={'train_runtime': 319.9533, 'train_samples_per_second': 93.792, 'train_steps_per_second': 11.73, 'total_flos': 497566386108288.0, 'train_loss': 1.300201029768633, 'epoch': 3.0})

In [7]:
distil3_eval = trainer.evaluate()

In [8]:
distil3_eval

{'eval_loss': 0.46652093529701233,
 'eval_accuracy': 0.8941558441558441,
 'eval_f1_macro': 0.8936446681976905,
 'eval_runtime': 7.1246,
 'eval_samples_per_second': 432.305,
 'eval_steps_per_second': 27.089,
 'epoch': 3.0}